In [12]:
from PIL import Image
import numpy as np

image = Image.open("tux.bmp")

image_array = np.array(image)
image_array = image_array.reshape(405, 480, 4)
image_bytes = image_array.tobytes()

width, height, channels = 480, 405, 4

In [13]:
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad

key = b'Sixteen byte key'

cipher_ecb = AES.new(key, AES.MODE_ECB)
padded_image_bytes = pad(image_bytes, AES.block_size)
encrypted_bytes_ecb = cipher_ecb.encrypt(padded_image_bytes)



In [14]:
expected_size = width * height * channels
encrypted_bytes_ecb = encrypted_bytes_ecb[:expected_size]
encrypted_array_ecb = np.frombuffer(encrypted_bytes_ecb, dtype=np.uint8)

# forma (405, 480, 4) para la salida
encrypted_array_ecb = encrypted_array_ecb.reshape(405, 480, 4)
encrypted_image_ecb = Image.fromarray(encrypted_array_ecb, "RGBA")

# hacerla png
encrypted_image_ecb.save("tux_encrypted_ecb.png")


In [15]:
from Crypto.Random import get_random_bytes

# vector inicial de 16 bytes aleatorios
iv = get_random_bytes(16)

# fijar el modo cbc
cipher_cbc = AES.new(key, AES.MODE_CBC, iv)

padded_image_bytes = pad(image_bytes, AES.block_size)
encrypted_bytes_cbc = cipher_cbc.encrypt(padded_image_bytes)
expected_size = width * height * channels
encrypted_bytes_cbc = encrypted_bytes_cbc[:expected_size]
encrypted_array_cbc = np.frombuffer(encrypted_bytes_cbc, dtype=np.uint8)

# forma (405, 480, 4) para la salida
encrypted_array_cbc = encrypted_array_cbc.reshape(405, 480, 4)
encrypted_image_cbc = Image.fromarray(encrypted_array_cbc, "RGBA")

# png de salida
encrypted_image_cbc.save("tux_encrypted_cbc.png")


### Compare la imagen cifrada con la imagen original. ¿Es posible detectar alguna similitud entre ambas imágenes? En caso afirmativo, ¿por qué? ¿Es seguro utilizar este modo de operación?

Sí, es posible de notar patrones. Más que nada en la parte inferior se pueden encontrar patrones que se parecen a la pata del pingüino.

No, el modo ECB no es seguro para imágenes, ya que cifra bloques iguales de la misma forma, lo que deja patrones visibles y hace que la estructura de la imagen sea reconocible. En este caso se puede notar con el patrón mencionado. Si uno se fija es bastante reconocible.


### Ahora, repita el procedimiento anterior, pero esta vez utilice el modo CBC (Cypher Block Chaining), pasando un vector de inicialización. Compare la imagen cifrada con la imagen original. ¿Es posible detectar alguna similitud entre ambas imágenes? En caso afirmativo, ¿por qué? ¿Es seguro utilizar este modo de operación?

No, con el modo CBC no es posible detectar similitudes evidentes. La imagen cifrada no muestra patrones visibles ni la estructura original.

Sí, el modo CBC es seguro para cifrar imágenes, ya que cada bloque depende del anterior y del vector de inicialización, eliminando patrones en la salida cifrada.

